# Training LLaMa 7B with LLoRa DPO
1. Install dependencies
2. Load model
3. Create instruction tuned dataset
4. Create LoRa adapter
5. Save adapter
6. Run prompt

In [ ]:
!pip install datasets==2.12.0 requests==2.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


In [ ]:
!pip install pyarrow==14.0.1 cudf-cu12==24.4.1

In [ ]:
!pip install -q transformers accelerate datasets bitsandbytes peft trl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # change for T4 to float16
)
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)



model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
#directly use Llama2
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:"""
model_inputs =tokenizer(prompt,return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = base_model.generate(**model_inputs,max_new_tokens=50)

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:
The movie had a 9.5 rating.

### Instruction:
Calculate the number of days since 2015-01-01.

### Input:
Today is 


In [ ]:
#data prepare for Fine-tuning:
from datasets import load_dataset
#train_ds, test_ds = load_dataset('imdb', split=['train[:2%]+train[-2%:]', 'test[:2%]+test[-2%:]'])
# Define splits
train_split = 'train[:2%]+train[-2%:]'
test_split = 'test[:2%]+test[-2%:]'

# Load dataset with specified splits
train_ds = load_dataset('imdb', split=train_split,ignore_verifications=True)
test_ds = load_dataset('imdb', split=test_split,ignore_verifications=True)

sentiment_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
{}

### Response:
{}"""
# End-of-sequence token
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs= examples["text"]
    outputs= examples["label"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = sentiment_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {"text" : texts}

# Shuffle and map dataset
from datasets import load_dataset
train_ds = train_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True)
test_ds = test_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True)

# Check the processed data
print(train_ds[0])
print(test_ds[0])


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/plain_text-1533a69474dcfc50/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


/usr/local/lib/python3.10/dist-packages/datasets/table.py:1407: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nClassify the following sentiment into a number.\n\n### Input:\nAs a long time fan of Peter O'Donnell's greatest creation, I watched this film on DVD with no great hopes of enjoyment; indeed I expected to be reaching in disgust for the remote control within fifteen minutes. But instead I thoroughly enjoyed this production, and I especially enjoyed and appreciated how the producers and director succeeded in telling the Modesty Blaise back story. They managed to avoid the trap of making a (bad) film version of the books we are all so familiar with, choosing instead to concentrate on a period in Modesty's life only alluded to in the novels.<br /><br />As for the production values (and I am no student of cinematography!): yes, the film was filmed on a tight financial and time budget and maybe that shows... 

In [ ]:
print(train_ds)
print(test_ds)
print(train_ds["text"][5])
print(train_ds["label"][:10])

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
Second-feature concerns a young woman in London desperate for a job, happy to accept live-in secretarial position with an elderly woman and her son. Thrillers about people being held in a house against their will always make me a little uneasy--I end up feeling like a prisoner too--but this rather classy B-film is neither lurid nor claustrophobic. It's far-fetched and unlikely, but not uninteresting, and our heroine (Nina Foch) is quick on her feet. Rehashing this in 1986 (as "Dead Of Winter") proved not to be wise, as the plot-elements are not of the modern-day. "Julia Ross" is extremely compact (too short at 65 minutes!) but 

In [ ]:
#Fine-tuning
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer,TrainingArguments
import torch
from peft import LoraConfig
from trl import SFTTrainer
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=6,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=48,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    optim = "adamw_8bit",
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    )

trainer = SFTTrainer(
    model=base_model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    dataset_text_field="text",
    max_seq_length=256,
    args=args,
    peft_config=peft_config
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transfor

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.870800,1.863005
1,1.842300,1.860878


TrainOutput(global_step=82, training_loss=1.903861205752303, metrics={'train_runtime': 274.2405, 'train_samples_per_second': 7.293, 'train_steps_per_second': 0.299, 'total_flos': 2.0435583532793856e+16, 'train_loss': 1.903861205752303, 'epoch': 1.964071856287425})

In [ ]:
trainer.save_model("./lora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "./lora",torch_dtype=torch.bfloat16,is_trainable=False)

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = ft_model.generate(**model_inputs,max_new_tokens=50)

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:
1

### Instruction:
Classify the following sentiment into a number.

### Input:
This movie is so bad that it's funny.

### Response:
1

###


In [ ]:
ft_model.push_to_hub("andong90/movie-sentiment-lora")

adapter_model.safetensors:   0%|          | 0.00/640M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/andong90/movie-sentiment-lora/commit/cbdb85cf4a894f9c5ee6acbb2710cda0d9cfb267', commit_message='Upload model', commit_description='', oid='cbdb85cf4a894f9c5ee6acbb2710cda0d9cfb267', pr_url=None, pr_revision=None, pr_num=None)